# Note: Please run the 'LoadTwb' notebook before running this notebook

##  This tool automates the extraction of metadata, from the tableau workbook, such as datasource,dimensions,measures, calcuated fields,parameters,etc. and load them into a standardized Excel workbook that will have sheets containing different types of data.

## A large chunk of the code performs extraction, trasnformation and loading (ETL) from the structured XML file (.twb file)

## This notebook excellently showcases use of list comprehensions, packages, nested for- loops,dictionaries and magic commands

##  Output path for the Excel workbook needs to be modified in the last cell.

In [ ]:
#Importing xml file (twb) into python

import xml.etree.ElementTree as ET
import pandas as pd
import re
from itertools import chain,zip_longest

%store -r fname
%load fname

tree = ET.parse(fname)
root = tree.getroot()

In [ ]:
#Declare variables

workbook_names , database_names, table_names, server_names, datasource_alias, dashboard_names,schema_names, worksheet_names = ([] for _ in range(8))


In [ ]:
#Get Datbase name

def get_database_names():
    
    database_names = []
    
    #if query_flag == 0:
        
    for i in root.iter('named-connection'):
        for j in i.iter('connection'):
            database_names.append(j.get('dbname',default=''))
        
    return database_names

In [ ]:
# Get Table names

def get_table_names():
    
    table_names = []
    
    #Table names
    for i in root.iter('relation'):
        table_names.append(i.get('table',default=''))
    
    
    table_names = [i.replace('[dbo].','') for i in table_names]
    table_names = [i.replace('dbo.','') for i in table_names]
    table_names = [i.replace('[','') for i in table_names]
    table_names = [i.replace(']','') for i in table_names]
    
    
    return table_names

In [ ]:
# Get server names

def get_server_names():
    
    server_names = []
    
    for i in root.iter('named-connection'):
        for j in i.iter('connection'):            
            server_names.append(j.get('server',default=''))
                
    return server_names

In [ ]:
#Get datasource alias

def get_datasource_alias():
    
    datasource_alias = []
    
    for i in root.iter('datasource'):
        
        if i.get('caption') is not None:
            datasource_alias.append(i.get('caption',default=''))
        
        else:
            datasource_alias.append(i.get('name',default=''))
                
            
    return datasource_alias

In [ ]:
# Get Dashboard names

def get_dashboard_names():
    
    dashboard_names = []
    
    for i in root.iter('dashboard'):
        dashboard_names.append(i.get('name',default=''))
    
    return dashboard_names

In [ ]:
#Get workbook name

def get_workbook_names():
    
    workbook_names = []
    
    fname_wo_ext = fname[:-4]  # remove twb

    slash_pos = fname_wo_ext.rfind('/')  # extract file name

    wrbk_name = fname_wo_ext[slash_pos+1:]
    
    workbook_names.append(wrbk_name)
        
    return workbook_names

In [ ]:
# Get worksheet name

def  get_worksheet_names():
    
    worksheet_names = []
    
    for i in root.iter('worksheet'):
        worksheet_names.append(i.get('name',default=''))
        
    return worksheet_names

## From here, the code essentially writes the extracted metadata into specific excel tabs/sheets

In [ ]:
# Executive Summary Tab

ex_desc = []
kpi = []
tableau_version = []
dashboard_published_not_published = []

#Dashboard names
dashboard_names = get_dashboard_names()    
#print(dashboard_names)

# workbook names
workbook_names = get_workbook_names()    
#print(workbook_names)

#Ex_Description  #Kpi #Tableau version
ex_desc.append('')
kpi.append('')
    
for i in root.iter('workbook'):    
    tableau_version.append(i.get('version',default=''))   

#print(tableau_version)

zip_list = zip_longest(workbook_names,dashboard_names,ex_desc,kpi,tableau_version,dashboard_published_not_published,
                      fillvalue = '')

df_exec_sumry = pd.DataFrame(zip_list, columns = ['Workbook','Dashboard','Ex_Description','Key_Performance_Indicators',
                                                  'Tableau_Version','Dashboard_Published_Non-Published'])

df_exec_sumry

In [ ]:

# Data Source tab

custom_sql_query = []
connection_type = []
sql_query_flag = 0
table_view = []
data_source_path = []
zip_list = []
t_names = []


    #Custom SQL query
for i in root.iter('connection'):
    for j in i.iter('relation'):
        if j.get('type') == 'text':
            sql_query_flag = 1  
            #print("Custom SQL exists!",'\n\n')
            custom_sql_query.append(j.text)

        else:   
            custom_sql_query.append('')

#Table names

table_names = get_table_names()
#print(table_names)


 #Database names

database_names = get_database_names()
#print(database_names)


#Server names  
server_names = get_server_names()
#print(server_names)

#Schema name

for i in root.iter('relation'):
    t_names.append(i.get('table',default=''))

for i in t_names:
    schema_names.append(re.findall('^(\[\w+\]).',i, flags=re.IGNORECASE))

schema_names = list(chain.from_iterable(schema_names))

schema_names = [i.replace('[','') for i in schema_names]
schema_names = [i.replace(']','') for i in schema_names]
#print(schema_names)

#Datasource caption
datasource_alias = get_datasource_alias()
#print(datasource_alias)


# workbook names
workbook_names = get_workbook_names()    
#print(workbook_names)

#Connection_Type
       



zip_list = zip_longest(workbook_names,server_names,database_names,schema_names,table_names,table_view,custom_sql_query,
                       connection_type,datasource_alias,data_source_path,fillvalue = '')

df_datasource = pd.DataFrame(zip_list, columns = ['Workbook','Server_Name','Database_Name','Schema_Name',
                                                  'Table_Name','Table_View','Custom_SQL','Connection_Type',
                                                  'Data_Source_Alias','Data_Source_Path'])

df_datasource = df_datasource.drop_duplicates()

df_datasource

In [ ]:
# Worksheet tab

used = []
worksheet_desc = []
unused_worksheet_names = []
all_worksheet_names = []
dashboard_names = []
worksheet_names = []

for i in root.iter('window'):
    
    if i.get('class') == 'dashboard':
        
        for j in i[0].iter('viewpoint'):                            # [] helps pick only specific child nodes
            dashboard_names.append(i.get('name',default=''))
            worksheet_names.append(j.get('name',default = ''))

            
for i in root.iter('worksheet'):
    all_worksheet_names.append(i.get('name',default=''))
    
all_worksheet_names = set(all_worksheet_names)
        

# workbook names
workbook_names = get_workbook_names()
    
zip_list = zip_longest(workbook_names,dashboard_names,worksheet_names,used,worksheet_desc,
                      fillvalue = '')

df_used = pd.DataFrame(zip_list, columns = ['Workbook','Dashboard','Worksheet','Used','Worksheet_Description'])

# Take non-matching names between used and all worksheets
unused_worksheet_names = all_worksheet_names.symmetric_difference(set(df_used['Worksheet']))

df_unused = pd.DataFrame(unused_worksheet_names,columns = ['Worksheet'])

df_worksheet = pd.concat([df_used,df_unused], sort = False).fillna('')

df_worksheet

In [ ]:
# Fields and Tables tab

# Datasource field names
datasource_field_names = []
tableau_field_names = []
measure_dimension = []
data_type = []
used = []
sql_calcs = []
original_field_name = []
tableau_fnames = []
local_field_names = []
local_field_type = []
meas_dim = []
dashboard_names = []
worksheet_temp_col = []



for i in root.iter('worksheet'):

    for col_info in i.iter('column'):
    
        if col_info.find('calculation') is None:
        
            if col_info.get('caption') is not None or col_info.get('name') is not None:

                if col_info.get('caption') is not None:

                    tableau_field_names.append(col_info.get('caption',default=''))

                else:
                    tableau_field_names.append(col_info.get('name',default=''))

                worksheet_temp_col.append(i.get('name',default = ''))    

                datasource_field_names.append(col_info.get('name',default=''))

                measure_dimension.append(col_info.get('role',default=''))

                data_type.append(col_info.get('datatype',default=''))


# workbook names 
workbook_names = get_workbook_names()

#Server names  
server_names = get_server_names()
            
#Table names    
table_names = get_table_names()


#Database names
database_names = get_database_names()


#Fixing missing fields    
for i in root.iter('metadata-record'):
    
    for j in i.iter('local-name'):
        
        local_field_names.append(j.text)

        
    for l in i.iter('local-type'):
        
        local_field_type.append(l.text)


for i in local_field_type:
    
    if i in ('string','datetime','boolean'):
        
        meas_dim.append('dimension')
    
    else:
        
        meas_dim.append('measure')


tableau_fnames = [i for i in local_field_names]        
# find missing elements and create a list


zip_list = zip_longest(workbook_names,dashboard_names,worksheet_temp_col,tableau_field_names,datasource_field_names,table_names,database_names,
               server_names,data_type,used,measure_dimension,sql_calcs,original_field_name,
                      fillvalue = '')

df1 = pd.DataFrame(zip_list, columns =  ['Workbook','Dashboard','Worksheet_Temp_Col','Tableau_Field_Name','Datasource_Field_Name',
                                                      'Table_Name','Database_Name','Server_Name','Data_Type','Used',
                                                      'Dimension_Measure','SQL_Calculations','Original_Field_Name'])


zip_union_list = zip_longest(tableau_fnames,local_field_names,meas_dim,local_field_type,fillvalue = '')

df2 = pd.DataFrame(zip_union_list,columns = ['Tableau_Field_Name','Datasource_Field_Name','Dimension_Measure','Data_Type'])

df2.drop_duplicates()

df_fields_tables = pd.concat([df1,df2],sort = False)

df_fields_tables = df_fields_tables.fillna('')


# Dashboard names


#  Fields used directly in dashboard
for row,col in df_fields_tables.iterrows():
               
    for a,b in df_worksheet.iterrows():
        
        if col['Worksheet_Temp_Col'] == b['Worksheet']:
            
            df_fields_tables.iat[row,1] = b['Dashboard']
            
            


df_fields_tables = df_fields_tables.drop(['Worksheet_Temp_Col'], axis = 1)
                        
    
# Fields used through calc formulas is performed in the end of cell for df_calc_fields   
    
     
# The below output is not final, part of operation is performed later
df_fields_tables

In [ ]:
#Calculated Fields tab


# ----------------------Capture calculated fields-------------------------------

calculated_field_names = []
calculated_field_formulas = []
measure_dimension = []
calc_ids = []
param_ids  = []
fields_used = []
used = []
calc_type = []

calculated_field_names_2 = []
calculated_field_formulas_2 = []
measure_dimension_2 = []
calc_ids_2 = []
param_ids_2 = []
fields_used_2 = []

calc_captions = {}
param_captions = {}

calc_captions_2 = {}
param_captions_2 = {}

dashboard_names = []
worksheet_temp_col = []

formulas_wo_comments = []
formulas_wo_comments_2 = []

#Function to unnest a list as a result of re operations and also filter unique values using set

def unnest_and_set(some_list):   
    unnested_lst = list(chain.from_iterable(some_list))

    unnested_lst_set = set(unnested_lst)

    lst_with_unique_elems = list(unnested_lst_set)
    
    return lst_with_unique_elems



#Capture contents of a calculated field on a worksheet basis

#-------- Making changes here to test if worksheet col needs to be added prior to

for i in root.iter('worksheet'):

    for col_info in i.iter('column'):

        #Avoid capturing parameters along with calc fields
        if col_info.find('calculation') is not None and col_info.get('param-domain-type') is None:  

            if col_info.get('caption'):
                
                worksheet_temp_col.append(i.get('name',default = ''))
                calculated_field_names.append(col_info.get('caption',default=''))

            else:
                
                worksheet_temp_col.append(i.get('name',default = ''))
                calculated_field_names.append(col_info.get('name',default=''))


            for formula_info in col_info.iter('calculation'):

                calculated_field_formulas.append(formula_info.get('formula',default=''))          

            measure_dimension.append(col_info.get('role',default=''))

        
# Capture calc fields not used in any worksheets aka unused calc fields     

for k in root.iter('column'):

        #Avoid capturing parameters along with calc fields
        if k.find('calculation') is not None and k.get('param-domain-type') is None:  

            if k.get('caption'):
                
                calculated_field_names_2.append(k.get('caption',default=''))

            else:
                
                calculated_field_names_2.append(k.get('name',default=''))

            for l in k.iter('calculation'):

                calculated_field_formulas_2.append(l.get('formula',default=''))          

            measure_dimension_2.append(k.get('role',default=''))
            
            
# Capture calculation ids inside of other calculations
for elems in calculated_field_formulas:
    
    calc_ids.append(re.findall('\[Calculation_.*?\]',elems))
    param_ids.append(re.findall('\[Parameter .*?\]',elems))    

calc_ids = unnest_and_set(calc_ids)
param_ids = unnest_and_set(param_ids)


for elems_2 in calculated_field_formulas_2:
    
    calc_ids_2.append(re.findall('\[Calculation_.*?\]',elems_2))
    param_ids_2.append(re.findall('\[Parameter .*?\]',elems_2))    

calc_ids_2 = unnest_and_set(calc_ids_2)
param_ids_2 = unnest_and_set(param_ids_2)





# Find the names of calculated fields corresponding to the calculation ids and create a dictionary off of it
for elem in calc_ids:
    
    for child in root.iter('column'):
        
        calc_name = child.get('name',default='')
        calc_caption = child.get('caption',default='')
        
        if calc_name == elem:
            
            calc_captions[elem] = calc_caption

# ------

for elem_2 in calc_ids_2:
    
    for child_2 in root.iter('column'):
        
        calc_name_2 = child_2.get('name',default='')
        calc_caption_2 = child_2.get('caption',default='')
        
        if calc_name_2 == elem_2:
            
            calc_captions_2[elem_2] = calc_caption_2
        
        
        
        
        
            
# Find the names of parameters corresponding to the paramerer ids and create a dictionary off of it
for elem in param_ids:
    
    for child in root.iter('column'):
        
        param_name = child.get('name',default='')
        param_caption = child.get('caption',default='')
        
        if param_name == elem:
            
            param_captions[elem] = param_caption
    

    
for elem_2 in param_ids_2:
    
    for child_2 in root.iter('column'):
        
        param_name_2 = child_2.get('name',default='')
        param_caption_2 = child_2.get('caption',default='')
        
        if param_name_2 == elem_2:
            
            param_captions_2[elem_2] = param_caption_2    
    
    
#print(param_captions,'\n\n')    
    
#Replace calculation ids with actual names using list comprehensions
for key in calc_captions:

    calculated_field_formulas = [strs.replace(key,calc_captions[key]) for strs in calculated_field_formulas]  


    
for key in param_captions:
    
    calculated_field_formulas = [strs.replace(key,param_captions[key]) for strs in calculated_field_formulas]    
    
    
for key in calc_captions_2:

    calculated_field_formulas_2 = [strs.replace(key,calc_captions_2[key]) for strs in calculated_field_formulas_2]  


    
for key in param_captions_2:
    
    calculated_field_formulas_2 = [strs.replace(key,param_captions_2[key]) for strs in calculated_field_formulas_2]

    
#Fields used

combo_list = []

#combo_list = tableau_field_names + datasource_field_names + calculated_field_names

combo_list = list(df_fields_tables['Tableau_Field_Name']) + list(df_fields_tables['Datasource_Field_Name']) + calculated_field_names





# Creating a new list of calc field formulas with // removed for scanning fields used accurately

for i in calculated_field_formulas:
    
    i = i.strip()
    
    if "//" in i:
        
        pos = i.index('//')
    
        formulas_wo_comments.append(i[:pos])
        
    else:
        
        formulas_wo_comments.append(i)

#---        
for j in calculated_field_formulas_2:
    
    j = j.strip()
    
    if "//" in j:
        
        pos2 = j.index('//')
    
        formulas_wo_comments_2.append(j[:pos2])
        
    else:
        
        formulas_wo_comments_2.append(j)



for text in formulas_wo_comments:

    fields_used.append([field for field in combo_list if field in text])

fields_used = [list(set(i)) for i in fields_used]

fields_used = [str(i) for i in fields_used]

fields_used = [i.replace(',',';') for i in fields_used]

#----

for text_2 in formulas_wo_comments_2:

    fields_used_2.append([field_2 for field_2 in combo_list if field_2 in text_2])

fields_used_2 = [list(set(i)) for i in fields_used_2]

fields_used_2 = [str(i) for i in fields_used_2]

fields_used_2 = [i.replace(',',';') for i in fields_used_2]


#Workbook names
workbook_names = get_workbook_names()



# Use zip function to create a dictionary of all the three columns
calc_fields = zip_longest(workbook_names,dashboard_names,worksheet_temp_col,calculated_field_names,measure_dimension,
                          calculated_field_formulas,fields_used,used,calc_type,
                          fillvalue = '')

df_calc_fields = pd.DataFrame(calc_fields, columns =  ['Workbook','Dashboard','Worksheet_Temp_Col','Calculated_Field_Name','Tableau_Field_Type',
                                                       'Metric_Description','Fields_Used','Used','Calculation_Type'])


calc_fields_2 = zip_longest(calculated_field_names_2,measure_dimension_2,
                           calculated_field_formulas_2,fields_used_2,
                           fillvalue = '')

df_calc_fields_2 = pd.DataFrame(calc_fields_2, columns =  ['Calculated_Field_Name','Tableau_Field_Type',
                                                           'Metric_Description','Fields_Used'])

df_calc_fields_2 = df_calc_fields_2.drop_duplicates(subset = ['Calculated_Field_Name'])
#Dashboard names

                            
for row,col in df_calc_fields.iterrows():
               
    for a,b in df_worksheet.iterrows():
        
        if col['Worksheet_Temp_Col'] == b['Worksheet']:
            
            df_calc_fields.iat[row,1] = b['Dashboard']
            
            


df_calc_fields = df_calc_fields.drop(['Worksheet_Temp_Col'], axis = 1)

# Combine dfs
df_calc_fields = pd.concat([df_calc_fields,df_calc_fields_2], sort = False).fillna('')


    # Used in calc_formulas

for row,col in df_calc_fields.iterrows():
    
    for a,b in df_calc_fields.iterrows():
        
        if col['Dashboard'] == '' and b['Dashboard']!= '':

            if col['Calculated_Field_Name'] in b['Fields_Used']:

                df_calc_fields.iat[row,1] = b['Dashboard']


df_calc_fields = df_calc_fields.drop_duplicates(subset = ['Dashboard','Calculated_Field_Name'])


# continued from df_fields_tables . Dashboard names for df_fields_tables

for row,col in df_fields_tables.iterrows():
    
    for a,b in df_calc_fields.iterrows():
        
        if b['Dashboard'] != '' and col['Dashboard'] == '':
            
            if col['Tableau_Field_Name'] in b['Fields_Used'] or col['Datasource_Field_Name'] in b['Fields_Used']:
                
                df_fields_tables.iat[row,1] = b['Dashboard']
                

df_fields_tables = df_fields_tables.drop_duplicates(subset = ['Dashboard','Datasource_Field_Name'])       

df_calc_fields = df_calc_fields.drop_duplicates(subset = ['Dashboard','Calculated_Field_Name'])

df_calc_fields

In [ ]:
# Filters tab

#Filter_name

filter_names_raw = []
filter_names_raw2 = []
filter_names = []
filter_names_full = []
calc_ids = []
some_list = []
calc_captions = {}
filter_action = []
fields_used = []
sheet_name = []
dashboard_names = []
worksheet_temp_col = []  # Capture worksheet names to further capture dashboard names, choose to keep or drop this column after the fact

#Worksheet filters

for i in root.iter('worksheet'):
    
    for j in i.iter('slices'):
    
        for k in j.iter('column'):
    
            filter_names_full.append(k.text)
            worksheet_temp_col.append(i.get('name',default = ''))



for i in filter_names_full:
    
    filter_names_raw.append(re.findall('.(\[.*\])',i))
    

filter_names_raw = list(chain.from_iterable(filter_names_raw))
#print(filter_names_raw)


for i in filter_names_raw:
    
    filter_names_raw2.append(re.sub('\[\w+:','[',i))


for i in filter_names_raw2:
    
    filter_names.append(re.sub(':\w+\]',']',i))

    
#print('Filter_names: ',filter_names,'\n\n')
    
for i in filter_names:
    
    calc_ids.append(re.findall('.*(\[Calculation_.*\]).*',i,flags = re.IGNORECASE))


calc_ids = unnest_and_set(calc_ids)

#print('Calc ids: ',calc_ids,'\n\n')

# Find the names of calculated fields corresponding to the calculation ids and create a dictionary off of it
for elem in calc_ids:
    
    for child in root.iter('column'):
        
        calc_name = child.get('name',default='')
        calc_caption = child.get('caption',default='')
        
        if calc_name == elem:
            
            calc_captions[elem] = calc_caption
            
            
#print('Calc captions: ',calc_captions,'\n\n')

        
for key in calc_captions:        

    filter_names = [strs.replace(key,calc_captions[key]) for strs in filter_names]
        

#filter_names = list(set(filter_names))
#print('Filter names: ',filter_names,'\n\n')    

#filter_names = [i.replace('[','') for i in filter_names]
#filter_names = [i.replace(']','') for i in filter_names]    


#Workbook names
workbook_names = get_workbook_names()


filter_action.append('Selected')
                            
                            
        
zip_list = zip_longest(workbook_names,dashboard_names,worksheet_temp_col,filter_names,filter_action,fields_used,sheet_name,
                   fillvalue = '')

df_filters = pd.DataFrame(zip_list, columns =  ['Workbook','Dashboard','Worksheet_Temp_Col','Filter_Name','Filter_Action','Fields_Used','Sheet_Name'])


#Dashboard names

#  Filters used directly in dashboard

for row,col in df_filters.iterrows():
               
    for a,b in df_worksheet.iterrows():
        
        if col['Dashboard'] == '' and b['Dashboard'] != '':
            
            if col['Worksheet_Temp_Col'] == b['Worksheet']:

                df_filters.iat[row,1] = b['Dashboard']


df_filters = df_filters.drop(['Worksheet_Temp_Col'], axis = 1)


df_filters = df_filters.drop_duplicates(subset = ['Dashboard','Filter_Name'])  #remove this if you want to capture data across worksheets and not just across workbook

df_filters       

In [ ]:
# Actions tab

action_names = []
source_sheet = []
target_sheet = []
run_action_on = []
target_filter = []
action_description = []
worksheet_dashboard = []
dashboard_names = []

#Action Data
for i in root.iter('action'):
    
    action_names.append(i.get('caption',default=''))

#target sheet and source sheet is giving duplicate data. need to fix this
    for j in i.iter('source'):
        
        source_sheet.append(j.get('worksheet',default=''))
        dashboard_names.append(j.get('dashboard', default=''))
    
    for j in i.iter('activation'):
        
        run_action_on.append(j.get('type',default=''))
    
    
#Workbook names
workbook_names = get_workbook_names()
   

# Use zip function to create a dictionary of all columns
zip_list = zip_longest(workbook_names,dashboard_names,action_names,source_sheet,target_sheet,run_action_on,target_filter,action_description,
                       worksheet_dashboard,fillvalue = '')

df_actions = pd.DataFrame(zip_list, columns =  ['Workbook','Dashboard','Action_Name','Source_Sheet',
                                                    'Target_Sheet','Run_Action_On','Target_Filter',
                                               'Action_Description','Worksheet_Dashboard'])

df_actions = df_actions.drop_duplicates()  
df_actions

In [ ]:
# Parameters

parameter_name = []
data_type = []
allowable_value_type = []
param_desc = []
fields_used = []
used = []
wrksht_temp_col = []

parameter_name_2 = []
data_type_2 = []
allowable_value_type_2 = []

for i in root.iter('worksheet'):

    for j in i.iter('column'):
    
        if j.get('param-domain-type') is not None:
        
            wrksht_temp_col.append(i.get('name',default=''))
            
            parameter_name.append(j.get('caption',default=''))
            data_type.append(j.get('datatype',default=''))
            allowable_value_type.append(j.get('param-domain-type',default=''))


            
for k in root.iter('column'):
    
    if k.get('param-domain-type') is not None:
        
        parameter_name_2.append(k.get('caption',default=''))
        data_type_2.append(k.get('datatype',default=''))
        allowable_value_type_2.append(k.get('param-domain-type',default=''))
        
        
#Workbook names
workbook_names = get_workbook_names()        


# Create df
zip_list = zip_longest(workbook_names, dashboard_names, wrksht_temp_col, parameter_name, data_type, allowable_value_type,param_desc,fields_used,used,
                      fillvalue = '')

df_parameters = pd.DataFrame(zip_list, columns = ['Workbook','Dashboard','Wrksht_Temp_Col','Parameter_Name','Data_Type',
                                                  'Allowable_Value_Type','Parameter_Description','Fields_Used','Used'])



zip_list_2 = zip_longest(parameter_name_2, data_type_2, allowable_value_type_2,
                         fillvalue = '')

df_parameters_2 = pd.DataFrame(zip_list_2, columns = ['Parameter_Name','Data_Type','Allowable_Value_Type'])

df_parameters_2 = df_parameters_2.drop_duplicates(subset = ['Parameter_Name'])

#Dashboard names


for row,col in df_parameters.iterrows():
    
    for a,b in df_worksheet.iterrows():
        
        if col['Wrksht_Temp_Col'] == b['Worksheet']:
            
            df_parameters.iat[row,1] = b['Dashboard']
            

df_parameters = df_parameters.drop(['Wrksht_Temp_Col'], axis = 1)

df_parameters = pd.concat([df_parameters,df_parameters_2], sort = False).fillna('')

df_parameters = df_parameters.drop_duplicates(subset = ['Dashboard','Parameter_Name'])

df_parameters

In [ ]:
# Refresh Schedule tab

ref_sched = []
ref_type = []

#Workbook names

# workbook names #Capture workbook names as many times as number of calc fields
workbook_names = get_workbook_names()
        
        
#Datasource caption
datasource_alias = get_datasource_alias()            
            

    
zip_list = zip_longest(workbook_names, ref_type, ref_sched, datasource_alias,
                      fillvalue = '')

df_refresh_schedule = pd.DataFrame(zip_list, columns = ['Workbook','Refresh_Type','Refresh_Schedule','Data_Source_Alias'])

df_refresh_schedule = df_refresh_schedule.drop_duplicates()

df_refresh_schedule

In [ ]:
#Unused fileds tab


#Workbook names

workbook_names = get_workbook_names()
        

#Datasource caption
datasource_alias = get_datasource_alias()

      
        
tableau_field_name = []
field_type = []

    
zip_list = zip_longest(workbook_names, datasource_alias, tableau_field_name, field_type,
                      fillvalue = '')

df_unused_fields = pd.DataFrame(zip_list, columns = ['Workbook','Data_Source_Alias','Tableau_Field_Name',
                                                  'Field_Type'])

df_unused_fields = df_unused_fields.drop_duplicates()

df_unused_fields

In [ ]:
#----------------- Write to excel---------------------

from openpyxl.styles import PatternFill
from openpyxl import load_workbook

excel_path = r'C:\Users............\\'    # Change this path to some local folder

excel_path += input("Enter output file name: ")

excel_path += '.xlsx'

df_exec_sumry.to_excel(excel_path, sheet_name = 'Executive Summary', index = False)

with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a') as writer:
    df_datasource.to_excel(writer, sheet_name = 'Data Source',index = False)        #Write to specific location
    df_worksheet.to_excel(writer, sheet_name = 'Worksheet',index = False)
    df_fields_tables.to_excel(writer, sheet_name = 'Fields and Tables',index = False)
    df_calc_fields.to_excel(writer, sheet_name = 'Calculated Fields',index = False)
    df_filters.to_excel(writer, sheet_name = 'Filters',index = False)
    df_actions.to_excel(writer, sheet_name = 'Actions',index = False)
    df_parameters.to_excel(writer, sheet_name = 'Parameters',index = False)
    df_refresh_schedule.to_excel(writer, sheet_name = 'Refresh Schedule',index = False)
    df_unused_fields.to_excel(writer, sheet_name = 'Unused Fields',index = False)
    

wb = load_workbook(excel_path)

sheet_names = wb.sheetnames

manual_cols = ['Ex_Description','Key_Performance_Indicators','Dashboard_Published_Non-Published',
               'Server_Name','Database_Name','Schema_Name','Table_View','Custom_SQL','Connection_Type',
               'Used','Worksheet_Description',
               'Table_Name','Database_Name','Server_Name','Used','SQL_Calculations','Original_Field_Name',
               'Metric_Description','Used','Calculation_Type',
               'Fields_Used','Sheet_Name',
               'Target_Sheet','Target_Filter','Action_Description','Worksheet_Dashboard',
               'Parameter_Description','Fields_Used','Used',
               'Refresh_Type','Refresh_Schedule','Data_Source_Alias',
               'Tableau_Field_Name','Field_Type']


for i in sheet_names:
    
    for cell in wb[i]['1']:

        if cell.value in manual_cols:

            cell.fill = PatternFill('solid', fgColor = "00af00")
            
        else:
            
            cell.fill = PatternFill('solid',fgColor = "00afff")

wb.save(excel_path)